## Семинар 2. Векторные представления

Сегодня мы поиграем с векторными представлениями:

1. обучим свои маленькие эмбеддинги (Word2Vec),

2. загрузим предобученные эмбеддинги (GloVe из `gensim` model zoo),

3. визуализируем вектора (PCA и t-SNE),

4. сделаем "мостик" к темам документов: от мешка слов к LDA.

Напоминание из лекции

- Дистрибутивная гипотеза: *похожие контексты → похожие значения*.

- Для контекстов мы часто задаём окно в ±k слов (или другой тип контекста).
- Два семейства подходов:

*count-based:* сначала считаем co-occurrence (матрица совместных встречаемостей), нормируем (PMI/PPMI), затем сжимаем (SVD/LSA);

*prediction-based:* учим эмбеддинги как параметры модели предсказания контекста (Word2Vec, SGNS).


В этом семинаре потрогаем оба, но акцент - на Word2Vec и визуализации.

Если вы запускаете локально, пригодится установить библиотеки:

In [ ]:
!pip install --upgrade nltk gensim bokeh

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
# скачать данные:
!wget https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1 -O ./quora.txt

# альтернативная ссылка: https://yadi.sk/i/BPQrUu1NaTduEw

**Загружаем корпус**

In [3]:
import numpy as np

with open("./quora.txt", encoding="utf-8") as file:
    data = list(file)

data[49]

'Should cricket be our national sport and not hockey?\n'

> **Что это за данные?** Это текстовый корпус, где каждая строка - отдельное "предложение/фраза". Мы будем трактовать каждую строку как мини-документ/последовательность токенов.

**Токенизация** - типичный первый шаг в NLP: переводим сырой текст в список токенов (слов и знаков препинания).

Здесь текст сырый: пунктуация и смайлы приклеены к словам, поэтому `str.split()` не подходит.

Возьмём `nltk` - библиотеку для частых NLP-задач (токенизация, стемминг, POS-теги и т.д.).
`WordPunctTokenizer` делит текст на последовательность буквенно-цифровых фрагментов и всего остального по регулярке `\w+|[^\w\s]+`.

In [6]:
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()

print(tokenizer.tokenize(data[32]))

['What', 'TV', 'shows', 'or', 'books', 'help', 'you', 'read', 'people', "'", 's', 'body', 'language', '?']


In [7]:
# Нужно:
# 1. привести всё к нижнему регистру
# 2. токенизировать каждую строку
# `data_tok` должен получиться списком списков токенов.

data_tok = [tokenizer.tokenize(row.lower()) for row in data]

In [8]:
print([' '.join(row) for row in data_tok[:2]])

["can i get back with my ex even though she is pregnant with another guy ' s baby ?", 'what are some ways to overcome a fast food addiction ?']


> Комментарий: мы делаем `.lower()` до токенизации, чтобы `Wine` и `wine` не считались разными словами.

### От co-occurrence к эмбеддингам: окно контекста, частоты, PMI/PPMI

Перед нейросетевыми эмбеддингами полезно пощупать руками count-based идею.

1) Окно контекста

Фиксируем окно $\pm k$ вокруг каждого слова. Например, при `window=2` для фразы:

> `я люблю пить чай`

для слова `пить` контекстом будут `люблю, чай` (и ещё соседние в пределах 2).

2) Co-occurrence-матрица

Мы можем построить матрицу $C(w,c)$: сколько раз слово $w$ встретилось рядом с контекстом $c$.

Ниже - мини-демо на маленьком словаре (чтобы не взрывать память):

In [9]:
from collections import Counter, defaultdict

window = 2  # размер окна контекста (±window)
min_count_demo = 50  # чтобы отфильтровать редкие слова и сделать демо компактным

# 1) считаем частоты слов
word_counts = Counter(tok for row in data_tok for tok in row)
vocab = {w for w, cnt in word_counts.items() if cnt >= min_count_demo}

# 2) строим co-occurrence C(w,c) только по словам из vocab
cooc = defaultdict(Counter)

for row in data_tok:
    row = [t for t in row if t in vocab]
    for i, w in enumerate(row):
        left = max(0, i - window)
        right = min(len(row), i + window + 1)
        for j in range(left, right):
            if j == i:
                continue
            c = row[j]
            cooc[w][c] += 1

# посмотрим пример
example_word = next(iter(vocab))
cooc[example_word].most_common(10)

[('panel', 323),
 ('installation', 320),
 ('in', 270),
 (',', 215),
 ('near', 173),
 ('service', 147),
 ('?', 75),
 ('internet', 42),
 ('san', 24),
 ('best', 24)]

In [10]:
example_word

'provider'

**PMI/PPMI**

PMI говорит, насколько пара (слово, контекст) встречается чаще, чем ожидалось бы случайно при независимости; PPMI обнуляет отрицательные PMI.

Если захочешь, мы можем в семинаре сделать отдельный блок PPMI+SVD как эмбеддинги и сравнить соседей с Word2Vec (это классный мостик count-based vs prediction-based).

### Нейросетевые эмбеддинги: Word2Vec (gensim)

**Word vectors:** есть много способов обучать эмбеддинги: Word2Vec, GloVe (разные функции потерь), fastText (учёт символов/субслов).

Начнём с простого: **`gensim`** - NLP-библиотека, где есть Word2Vec и другие векторные модели.
В `gensim` Word2Vec включает CBOW и skip-gram, а также hierarchical softmax / negative sampling.

In [ ]:
from gensim.models import Word2Vec

model = Word2Vec(
    data_tok,
    vector_size=32, # размерность эмбеддинга (d)
    min_count=5, # игнорировать слова, которые встретились < 5 раз (снижаем шум/словарь)
    window=5 # контекст: окно ±5 слов вокруг целевого
).wv

# .wv = KeyedVectors: удобный интерфейс для запросов по словам

> Что здесь происходит
>
> * Мы задаём окно `window=5` → это реализация "похожий контекст → похожее значение".
> * Модель оптимизирует параметры (эмбеддинги) через задачу предсказания контекста/слова (prediction-based).

In [12]:
# Доступ к векторам
model.get_vector('anything')

array([-3.2703798 ,  0.89271075,  0.57808965,  2.7331312 ,  0.6117832 ,
        1.9834588 ,  2.2986047 , -5.0459604 , -0.21637765,  2.2083833 ,
       -0.98380226,  1.4347664 ,  2.943324  ,  1.3266299 ,  2.763696  ,
       -1.2521437 , -1.9594982 , -0.9693537 ,  1.5585054 , -1.0123    ,
       -0.41538823, -1.1144416 , -0.18905485,  0.23876081, -0.1728048 ,
       -2.9270272 ,  1.0152389 ,  3.539622  ,  3.1948426 ,  1.3303949 ,
        0.29707754, -1.6811348 ], dtype=float32)

In [13]:
# Поиск похожих слов
model.most_similar('anything')

[('something', 0.9328370094299316),
 ('everything', 0.8739420175552368),
 ('nothing', 0.8731029033660889),
 ('everyone', 0.8160327076911926),
 ('nobody', 0.7649103403091431),
 ('anybody', 0.7597917914390564),
 ('him', 0.7218764424324036),
 ('hope', 0.7004485726356506),
 ('them', 0.6935770511627197),
 ('anyone', 0.6905859112739563)]

### Используем предобученную модель (GloVe через gensim model zoo)

Хорошая новость: можно загрузить предобученные эмбеддинги в пару строк.

gensim.downloader умеет смотреть список доступных моделей и загружать их по имени (`api.info()`, `api.load(...)`).

In [14]:
import gensim.downloader as api
model = api.load('glove-twitter-100')

[==================================================] 100.0% 387.1/387.1MB downloaded


In [15]:
# Поиск похожих слов
model.most_similar('wine')

[('beer', 0.8526083827018738),
 ('bottle', 0.7869870662689209),
 ('drink', 0.7679266333580017),
 ('tasting', 0.7584347724914551),
 ('coffee', 0.7540070414543152),
 ('drinks', 0.7457992434501648),
 ('drinking', 0.7210016846656799),
 ('beers', 0.7155925631523132),
 ('whiskey', 0.7105889320373535),
 ('wines', 0.708965539932251)]

In [16]:
model.most_similar(positive=["coder", "money"], negative=["brain"])

[('broker', 0.5820155739784241),
 ('bonuses', 0.5424473881721497),
 ('banker', 0.5385112762451172),
 ('designer', 0.5197198390960693),
 ('merchandising', 0.4964233338832855),
 ('treet', 0.4922019839286804),
 ('shopper', 0.4920562207698822),
 ('part-time', 0.4912828207015991),
 ('freelance', 0.4843311905860901),
 ('aupair', 0.4796452820301056)]

**GloVe vs Word2Vec:**

* Word2Vec - prediction-based (учим через локальное предсказание).
* GloVe - глобальная модель, опирающаяся на co-occurrence статистику (ближе к count-based миру).

На практике обе дают хорошие эмбеддинги, но по-разному приближаются к одной идее.

### Учёт морфологии и OOV: FastText

У Word2Vec и GloVe есть существенный недостаток: они воспринимают слова как атомарные единицы. Если слова `apple` и `apples` встретились в разных контекстах, модель не знает, что у них общий корень. Если мы встретим слово, которого не было в обучении (например, опечатку `helpp`), Word2Vec выдаст ошибку (KeyError).

[FastText](https://github.com/facebookresearch/fastText) (от Facebook AI Research) решает это, разбивая слова на n-граммы символов.
Например, для слова `apple` и n=3 это будут: `<ap, app, ppl, ple, le>`.

Вектор слова складывается из векторов его n-грамм. Это позволяет:
1. Понимать морфологию (суффиксы, корни).
2. Строить вектора для слов, которых не было в обучении (OOV), собирая их из знакомых кусочков.

In [ ]:
from gensim.models import FastText

# обучаем FastText на наших данных
# параметры похожи на Word2Vec, но под капотом учим n-граммы
ft_model = FastText(
    data_tok,
    vector_size=32,
    window=5,
    min_count=5
)

Проверка на OOV (Out Of Vocabulary). Придумаем слово, которого точно нет в корпусе Quora (например, с опечаткой или редкий неологизм)

In [44]:
oov_word = "neuralnetworks"

if oov_word in model.key_to_index:
    print(f"Слово '{oov_word}' есть в Word2Vec/GloVe словаре.")
else:
    print(f"Слова '{oov_word}' нет в Word2Vec/GloVe.")

Слова 'neuralnetworks' нет в Word2Vec/GloVe.


In [37]:
# а FastText справится
vector_oov = ft_model.wv[oov_word]
print(f"Вектор для '{oov_word}': {vector_oov[:5]}...")

Вектор для 'neuralnetworks': [ 0.9989384   0.20913406  0.18407343 -0.08604914  1.2928529 ]...


Попробуем найти соседей для слова с явной ошибкой.

In [42]:
print(ft_model.wv.most_similar("inteligence"))

[('intelligence', 0.9724630117416382), ('association', 0.9532545208930969), ('progressivism', 0.9487292766571045), ('assertiveness', 0.9460233449935913), ('depersonalization', 0.945956826210022), ('orientation', 0.9391425848007202), ('progression', 0.9378568530082703), ('municipality', 0.9374908208847046), ('diligence', 0.9369297027587891), ('interpolation', 0.9367811679840088)]


### Визуализация эмбеддингов (понижение размерности)

Мы хотим посмотреть, что получилось. Но вектора в 100D (или 30D+) - это не для человеческого глаза. Поэтому делаем **dimensionality reduction**.

Построим картинку для 1000 самых частых слов:

In [38]:
words = model.index_to_key[:1000]

print(words[::100])

['<user>', '_', 'please', 'apa', 'justin', 'text', 'hari', 'playing', 'once', 'sei']


In [39]:
# for each word, compute it's vector with model
word_vectors = np.stack([model.get_vector(word) for word in words], axis=0)

In [40]:
word_vectors.shape

(1000, 100)

In [41]:
assert isinstance(word_vectors, np.ndarray)
assert word_vectors.shape == (len(words), 100)
assert np.isfinite(word_vectors).all()

#### Линейная проекция: PCA

PCA - простейший линейный метод снижения размерности.

Геометрически PCA выбирает оси (главные компоненты), вдоль которых дисперсия максимальна.
Важно: в `sklearn` PCA реализуется через SVD на центрированных данных.

<img src="https://numxl.com/wp-content/uploads/principal-component-analysis-pca-featured.png" style="width:350">


Под капотом PCA пытается приблизить матрицу $X$ матрицей низкого ранга (в терминах линейного преобразования), минимизируя MSE:

$$
|(X W) \hat{W} - X|^2_2 \to_{W, \hat{W}} \min
$$

* $X \in \mathbb{R}^{n \times m}$ - матрица объектов (центрированная),
* $W \in \mathbb{R}^{m \times d}$ - прямое преобразование,
* $\hat{W} \in \mathbb{R}^{d \times m}$ - обратное преобразование,
* $n$ объектов, $m$ исходная размерность, $d$ - целевая.



In [21]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Проецируем векторы слов на плоскость (2D) с помощью PCA.
# Используем стандартный API sklearn (fit, transform)
pca = PCA(n_components=2)
word_vectors_pca = pca.fit_transform(word_vectors)

# После этого нормализуем векторы, чтобы получить нулевое среднее и единичную дисперсию
scaler = StandardScaler()
word_vectors_pca_scaled = scaler.fit_transform(word_vectors_pca)

In [22]:
assert word_vectors_pca_scaled.shape == (len(word_vectors), 2), "для каждого слова должен получиться 2D-вектор"
assert max(abs(word_vectors_pca_scaled.mean(0))) < 1e-5, "точки должны быть центрированы"
assert max(abs(1.0 - word_vectors_pca_scaled.std(0))) < 1e-2, "точки должны иметь единичную дисперсию"

#### Рисуем интерактивно

In [23]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ Рисует интерактивный график точек данных с дополнительной информацией при наведении курсора """
    if isinstance(color, str): color = [color] * len(x)
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig

In [24]:
# Обратите внимание: лучше рисовать scaled (нормализованные) векторы, чтобы график был красивее
draw_vectors(word_vectors_pca_scaled[:, 0], word_vectors_pca_scaled[:, 1], token=words)

# Наведите мышку и попробуйте увидеть «кластеры»: числа, эмоции, животные, профессии и т.д.

figure(id='p1011', ...)

> Комментарий: PCA показывает глобальную структуру (крупные направления вариации), но может терять локальные соседства.

### Визуализация соседей с t-SNE

PCA линейный. Если мы хотим лучше сохранить локальные соседства, смотрим в сторону t-SNE.

Очень важная заметка: t-SNE легко перечитать неправильно; полезно держать в голове типичные ловушки (см. [Distill-статью](https://distill.pub/2016/misread-tsne/)).

In [25]:
from sklearn.manifold import TSNE

# проецируем векторы слов на плоскость (2D) с помощью t-SNE.
# подсказка: не пугайтесь, процесс обучения может занять пару минут.

# word_vectors
word_vectors_tsne = TSNE(n_components=2).fit_transform(word_vectors)

# нормализуем векторы, так же как делали это для PCA
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
word_vectors_tsne_scaled = scaler.fit_transform(word_vectors_tsne)

In [26]:
draw_vectors(word_vectors_tsne_scaled[:,0], word_vectors_tsne_scaled[:,1], token=words)

figure(id='p1064', ...)

> Добавление: можно потом построить `draw_vectors(word_vectors_tsne_scaled[:,
0], word_vectors_tsne_scaled[:,1], token=words)` и сравнить, чем t-SNE отличается от PCA.


### От слов к документам: идеи тематического моделирования

Теперь мостик: эмбеддинги слов - это уровень лексики. А если мы хотим понять, о чём документы, нам нужен уровень тем.

> * Документ $\approx$ смесь тем $\theta_d$
> * Тема $\approx$ распределение по словам $\phi_k$
  Это концептуально соответствует тому, что мы обсуждали для LDA.

### LDA на практике (sklearn): выделяем темы в корпусе

В sklearn есть `LatentDirichletAllocation` (вариационный байес, online VB) и параметры вроде:

* n_components - число тем,
* doc_topic_prior (alpha) и topic_word_prior (beta).

**Готовим документы и мешок слов**

Возьмём первые N строк как документы (можно менять N для скорости).

In [27]:
from sklearn.feature_extraction.text import CountVectorizer

N_DOCS = 20000  # можете уменьшить, если долго
docs = [row.strip().lower() for row in data[:N_DOCS]]

# CountVectorizer строит матрицу "документ × слово" (мешок слов).
# Здесь можно добавить стоп-слова, min_df/max_df и т.д.
vectorizer = CountVectorizer(
    min_df=5,   # игнорируем слова, которые встретились < 5 документов
    max_df=0.5  # игнорируем слишком частые слова (в половине документов и больше)
)

X = vectorizer.fit_transform(docs)
X.shape

(20000, 3979)

> Комментарий: LDA любит мешок слов (counts). TF-IDF чаще связывают с LSA/NMF, хотя запустить можно всё, вопрос интерпретации.

**Обучаем LDA**

In [28]:
from sklearn.decomposition import LatentDirichletAllocation

n_topics = 10

lda = LatentDirichletAllocation(
    n_components=n_topics,
    random_state=42,
    learning_method="batch"  # можно попробовать "online"
)

doc_topic = lda.fit_transform(X)  # (N_DOCS, n_topics): распределение тем по документам
doc_topic.shape

(20000, 10)

>* `lda.components_[k]` - веса слов в теме k (до нормировки).
> * `doc_topic[d]` - доли тем в документе d (в сумме около 1).

**Смотрим топ-слова тем**

In [29]:
import numpy as np

feature_names = np.array(vectorizer.get_feature_names_out())

def print_top_words(model, feature_names, n_top_words=12):
    for topic_idx, topic in enumerate(model.components_):
        top_ids = topic.argsort()[-n_top_words:][::-1]
        top_words = feature_names[top_ids]
        print(f"Тема {topic_idx:02d}: " + ", ".join(top_words))

print_top_words(lda, feature_names)

Тема 00: to, if, would, it, you, and, be, for, how, that, will, in
Тема 01: the, of, you, what, do, did, why, think, will, is, have, and
Тема 02: is, the, what, and, between, difference, does, on, it, there, mean, to
Тема 03: are, why, in, to, what, do, and, people, some, they, how, of
Тема 04: can, how, in, to, get, do, should, for, where, we, be, an
Тема 05: the, what, is, of, in, to, best, are, and, way, most, for
Тема 06: the, what, are, for, best, in, which, is, some, of, and, good
Тема 07: is, it, to, what, why, or, for, good, like, better, at, an
Тема 08: how, do, my, you, can, on, to, make, with, get, your, in
Тема 09: how, does, the, do, why, in, on, to, of, and, can, quora


> А что случилось?

Проблема понятна: Stop words (стоп-слова). LDA - это же алгоритм, основанный на частотности. Если не убрать слова-связки ("the", "is", "to", "what"), они будут доминировать во всех темах, потому что они есть в каждом документе. Давайте просто зменим параметры `CountVectorizer`.

In [45]:
from sklearn.feature_extraction.text import CountVectorizer

N_DOCS = 20000
docs = [row.strip().lower() for row in data[:N_DOCS]]

# 1. stop_words='english' уберет "the", "is", "at", "which" и т.д.
# 2. max_df=0.1 уберет слова, которые встречаются чаще, чем в 10% вопросов
vectorizer = CountVectorizer(
    stop_words='english',
    min_df=10,
    max_df=0.1
)

X = vectorizer.fit_transform(docs)
print(f"Размер словаря: {len(vectorizer.get_feature_names_out())}")
X.shape

Размер словаря: 1974


(20000, 1974)

In [46]:
# с количеством топиков и n_top_words можно экспериментровать
n_topics = 10

lda = LatentDirichletAllocation(
    n_components=n_topics,
    random_state=42,
    learning_method="batch"
)

doc_topic = lda.fit_transform(X)

feature_names = vectorizer.get_feature_names_out()
print_top_words(lda, feature_names)

Тема 00: does, work, mean, better, sex, job, examples, water, english, good
Тема 01: does, long, read, bad, don, video, people, clinton, car, hillary
Тема 02: account, facebook, money, black, stop, phone, important, bank, marketing, white
Тема 03: like, know, life, day, person, new, things, going, people, real
Тема 04: make, old, year, company, movie, love, thing, years, google, think
Тема 05: best, way, learn, start, online, buy, programming, business, website, learning
Тема 06: quora, indian, girl, number, questions, live, like, question, girls, tell
Тема 07: india, time, trump, country, donald, china, make, president, people, feel
Тема 08: good, difference, engineering, student, computer, science, prepare, data, men, women
Тема 09: did, use, used, world, people, improve, earth, tv, create, energy


> Посмотрите на темы 00 и 09. Там всё ещё много слов вроде `does`, `did`, `make`. Это типичная ситуация в NLP: стандартного списка стоп-слов (`stop_words='english'`) часто мало.
>
> Как улучшить модель в реальной задаче? Можно расширить список стоп-слов, добавив туда специфичные для вопросников глаголы: my_stop_words = text.ENGLISH_STOP_WORDS.union(['does', 'did', 'make', 'know']). И переобучить модель. Вы можете самостоятельно с этим поэкспериментировать.

In [49]:
# попробуем вручную сформировать темы по словам
# некоторые темы могут быть очень приблизительными

topic_labels = {
    0: "Work & Life Meanings",    # work, job, mean, sex
    1: "Politics & Media",        # clinton, video, read
    2: "Finance & Social Media",  # money, bank, facebook, marketing
    3: "Life & Philosophy",       # life, people, real, things
    4: "Business & Tech Giants",  # company, google, apple
    5: "Education & Programming", # learn, programming, online, course
    6: "Quora Community",         # quora, questions, ask
    7: "Geopolitics",             # india, trump, china, president
    8: "STEM Education",          # engineering, science, student, data
    9: "Global Issues",           # world, earth, energy
}

In [54]:
# можно оценить теперь темы конкретного документа
doc_id = 31
top_topics = doc_topic[doc_id].argsort()[::-1][:3]

print(f"Текст вопроса:\n{docs[doc_id]}\n")
print("О чем этот текст (по мнению модели):")

for t in top_topics:
    if doc_topic[doc_id, t] > 0.1:
        label = topic_labels.get(t, f"Тема {t}")
        print(f" -> {label} (доля: {doc_topic[doc_id, t]:.1%})")

Текст вопроса:
are drivers within rich countries less likely to slow down when they see a driver's turn signal?

О чем этот текст (по мнению модели):
 -> Politics & Media (доля: 49.5%)
 -> Geopolitics (доля: 27.8%)
 -> Global Issues (доля: 14.9%)


### Интерактивная визуализация тем: pyLDAvis

Если хочется красивый интерактив (двумерная карта тем + слайдер λ), используется `pyLDAvis`. У него есть API и режим работы прямо в ноутбуке.

> Этот блок опционален: может потребовать установки `pyLDAvis`.

In [ ]:
!pip install pyldavis

Что будем туда передавать? Нам потребуются:

> `lda`: обученная sklearn LatentDirichletAllocation
>
> `X`: doc-term матрица, shape = (n_docs, n_terms)
>
> `vectorizer`: тот же CountVectorizer, на котором делали

In [64]:
import numpy as np
import pyLDAvis
pyLDAvis.enable_notebook()

# распределения слов по темам (нормируем строки)
topic_term = lda.components_.astype(float)
topic_term_dists = topic_term / topic_term.sum(axis=1, keepdims=True)

# распределения тем по документам
doc_topic_dists = lda.transform(X)

# длины документов (число токенов/термов)
# для sparse матрицы получится матрица (n_docs,1) — приводим к 1D
doc_lengths = np.asarray(X.sum(axis=1)).ravel()

# список терминов (в порядке колонок X)
vocab = vectorizer.get_feature_names_out()

# суммарная частота термов по корпусу
term_frequency = np.asarray(X.sum(axis=0)).ravel()

vis = pyLDAvis.prepare(
    topic_term_dists=topic_term_dists,
    doc_topic_dists=doc_topic_dists,
    doc_lengths=doc_lengths,
    vocab=vocab,
    term_frequency=term_frequency,
    sort_topics=False
)

In [65]:
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.067734 -0.237333       1        1  10.570292
1     -0.047268 -0.211345       2        1   8.967271
2     -0.046930  0.190939       3        1   8.736907
3     -0.162185 -0.039604       4        1  10.535336
4     -0.025207  0.143300       5        1  10.097234
5      0.312317  0.042522       6        1  11.027172
6     -0.113833  0.144357       7        1   8.462467
7     -0.154110 -0.059659       8        1  10.662122
8      0.208328  0.029273       9        1  11.205382
9     -0.038847 -0.002449      10        1   9.735815, topic_info=           Term         Freq        Total Category  logprob  loglift
195        best  1486.000000  1486.000000  Default  30.0000  30.0000
529        does  1363.000000  1363.000000  Default  29.0000  29.0000
898       india   668.000000   668.000000  Default  28.0000  28.0000
505         did   422.000000   422.000000  Default  27.0000  27.0000
1028       like   618.000000   618.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
266        card    42.414815    91.398121  Topic10  -5.1132   1.5616
1155     mobile    41.498938    92.207376  Topic10  -5.1350   1.5310
1395    product    34.691548    49.362546  Topic10  -5.3142   1.9767
1361   possible    37.525332   175.200651  Topic10  -5.2357   0.7884
414   countries    34.824706    74.193468  Topic10  -5.3104   1.5730

[538 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
1         5  0.504872       10
1         7  0.066681       10
1         9  0.419139       10
6         5  0.052810       12
6         9  0.924173       12
...     ...       ...      ...
1967      5  0.710258    years
1967      8  0.005870    years
1967     10  0.023480    years
1972      1  0.939828  youtube
1972      2  0.051497  youtube

[893 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

### Общий вывод:

* **Word2Vec**: учим эмбеддинги через предсказание контекстов (prediction-based).
* **GloVe / PMI / PPMI / SVD(LSA)**: учим/получаем представления из глобальной статистики co-occurrence (count-based).
* **LDA**: поднимаемся на уровень документов и выделяем темы (смеси тем по документам и распределения слов по темам).